In [21]:
import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [22]:
def custom_dist(u, v):
    if u[2] == v[2]:
        return sys.float_info.max
    else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0075,
                      'NoCurbRamp': 0.0075,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [23]:
df = gpd.read_file('data/processed-labels/q4.shp')

In [24]:
df

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.625000,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.33279 47.61860)
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.875000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.33189 47.61781)
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.500000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183 47.61751)
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.500000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33188 47.61752)
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.000000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33205 47.61764)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51290,84976,CurbRamp,1.0,-ckGXVaLK1TcDDw6tazt5Q,1,11.375000,-15.125000,310.992859,-0.179024,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615276,-122.339600,POINT (-122.33960 47.61528)
51291,84978,CurbRamp,1.0,-ckGXVaLK1TcDDw6tazt5Q,1,278.750000,-21.687500,310.992859,-0.179024,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615166,-122.339836,POINT (-122.33984 47.61517)
51292,84989,CurbRamp,1.0,d4y9dqjILO2W1rD6gJeWdA,1,333.062500,-31.625000,44.068935,1.698112,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615898,-122.341118,POINT (-122.34112 47.61590)
51293,84993,CurbRamp,1.0,kc_ZtnbGc7SLcZLOXdSb3w,2,8.241072,-8.214286,311.037811,-0.420952,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.616055,-122.340942,POINT (-122.34094 47.61605)


In [25]:
df0 = cluster_label_type_at_index(df,0)[2]

In [26]:
df1 = cluster_label_type_at_index(df,1)[2]

In [27]:
df2 = cluster_label_type_at_index(df,2)[2]

In [28]:
df3 = cluster_label_type_at_index(df,3)[2]

In [29]:
df4 = cluster_label_type_at_index(df,4)[2]

In [30]:
df5 = cluster_label_type_at_index(df,5)[2]

In [31]:
df6 = cluster_label_type_at_index(df,6)[2]

In [32]:
df7 = cluster_label_type_at_index(df,7)[2]

In [33]:
df8 = cluster_label_type_at_index(df,8)[2]

In [34]:
df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
df1["clustered"] = df1.duplicated(subset="cluster_id", keep=False)
df2["clustered"] = df2.duplicated(subset="cluster_id", keep=False)
df3["clustered"] = df3.duplicated(subset="cluster_id", keep=False)
df4["clustered"] = df4.duplicated(subset="cluster_id", keep=False)
df5["clustered"] = df5.duplicated(subset="cluster_id", keep=False)
df6["clustered"] = df6.duplicated(subset="cluster_id", keep=False)
df7["clustered"] = df7.duplicated(subset="cluster_id", keep=False)
df8["clustered"] = df8.duplicated(subset="cluster_id", keep=False)

In [35]:
df0

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id,clustered
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.625000,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.33279 47.61860),11574,False
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.500000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183 47.61751),3694,True
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.500000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33188 47.61752),5365,False
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.000000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33205 47.61764),1258,False
5,85068,CurbRamp,1.0,qzzOihXh8f6fpAKK9ttmUQ,1,240.312500,-28.437500,220.271088,-0.452202,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.616974,-122.332909,POINT (-122.33291 47.61697),11044,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51289,84971,CurbRamp,1.0,jNvtM4tQxRDEdrgyByIV_Q,1,274.812500,-14.937500,310.825928,0.078766,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615040,-122.339638,POINT (-122.33964 47.61504),13,True
51290,84976,CurbRamp,1.0,-ckGXVaLK1TcDDw6tazt5Q,1,11.375000,-15.125000,310.992859,-0.179024,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615276,-122.339600,POINT (-122.33960 47.61528),12,True
51291,84978,CurbRamp,1.0,-ckGXVaLK1TcDDw6tazt5Q,1,278.750000,-21.687500,310.992859,-0.179024,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615166,-122.339836,POINT (-122.33984 47.61517),11,True
51292,84989,CurbRamp,1.0,d4y9dqjILO2W1rD6gJeWdA,1,333.062500,-31.625000,44.068935,1.698112,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.615898,-122.341118,POINT (-122.34112 47.61590),2,True


In [36]:
#concatenate all dataframes
clusters_q4 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [37]:
clusters_q4

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id,clustered
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.75000,-15.625000,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.33279 47.61860),11574,False
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.18750,-24.500000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183 47.61751),3694,True
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.18750,-24.500000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33188 47.61752),5365,False
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.18750,-35.000000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33205 47.61764),1258,False
5,85068,CurbRamp,1.0,qzzOihXh8f6fpAKK9ttmUQ,1,240.31250,-28.437500,220.271088,-0.452202,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.616974,-122.332909,POINT (-122.33291 47.61697),11044,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18516,200557,Signal,NaN,QLvIv-d6zujBLdz3qd8PIA,1,350.62500,-3.937500,179.762634,0.250763,b98e5db3-e835-427a-a5fd-2d8fa43a166c,47.553463,-122.329735,POINT (-122.32973 47.55346),3,False
26662,200069,Signal,NaN,sfsTWWAxqByQHZIKHgJB3g,3,272.09375,-1.203125,269.002991,-0.082222,b98e5db3-e835-427a-a5fd-2d8fa43a166c,47.553486,-122.337051,POINT (-122.33705 47.55349),7,False
26738,202840,Signal,NaN,VHs7R8IAJCdgdxpN6wt_Pg,3,261.43750,-14.866072,248.595398,0.266441,ae9d3fab-6a0e-41c0-863e-ecff8ed43c7b,47.546303,-122.326363,POINT (-122.32636 47.54630),1,False
34230,202843,Signal,NaN,bK3iu1fIvMFQo95IWKTpxw,3,172.84375,-6.991071,248.698990,-0.402191,ae9d3fab-6a0e-41c0-863e-ecff8ed43c7b,47.546188,-122.326012,POINT (-122.32601 47.54619),4,False


In [38]:
#export to shapefile
clusters_q4.to_file("data/processed-labels/q4_clusters.shp")